<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-28 17:03:24


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[mypf['Criteria'].isin(['V40','V40N'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 73


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.drop('TTM', axis=1, inplace=True)
cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023']
for col in cols:
    df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

3MINDIA
5PAISA
ABBOTINDIA
AKZOINDIA
ANGELONE
ASIANPAINT
ASTRAZEN
AXISBANK
BAJAJ-AUTO
BAJAJFINSV
BAJAJHLDNG
BAJFINANCE
BATAINDIA
BAYERCROP
BERGEPAINT
BOSCHLTD
CAPLIPOINT
CERA
COLPAL
DABUR
DIXON
EICHERMOT
EQUITASBNK
ERIS
FINCABLES
FINEORG
GLAXO
HAVELLS
HCLTECH
HDFCAMC
HDFCBANK
HDFCLIFE
HINDUNILVR
HONAUT
ICICIBANK
ICICIGI
ICICIPRULI
INDIGOPNTS
INFY
ISEC
ITC
JCHAC
KANSAINER
KOTAKBANK
LALPATHLAB
LUXIND
MARICO
MCDOWELL-N
MCX
MOTILALOFS
NAM-INDIA
OFSS
PAGEIND
PFIZER
PIDILITIND
POLYCAB
RADICO
RAJESHEXPO
RELAXO
RELIANCE
SFL
SIS
SUNTV
SYMPHONY
TATAELXSI
TCS
TEAMLEASE
TITAN
TTKPRESTIG
UJJIVANSFB
VINATIORGA
VIPIND
WHIRLPOOL


In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='7y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[stock_df['Year'].str.contains("Mar")]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
0,Close,11084.33,18601.89,23247.37,18041.43,29165.19,18874.46,22801.62,3MINDIA.NS
0,Close,NaN,203.15,144.82,113.15,249.70,344.70,283.00,5PAISA.NS
0,Close,4350.48,5157.93,6919.31,14684.17,14483.08,17264.89,21794.16,ABBOTINDIA.NS
0,Close,1653.42,1567.10,1617.28,2013.28,2122.52,1826.17,2266.63,AKZOINDIA.NS
0,Close,NaN,NaN,NaN,NaN,268.62,1466.12,1136.93,ANGELONE.NS
...,...,...,...,...,...,...,...,...,...
0,Close,418.32,441.52,667.82,474.70,712.50,817.36,693.90,TTKPRESTIG.NS
0,Close,NaN,NaN,NaN,26.22,29.29,14.21,24.25,UJJIVANSFB.NS
0,Close,363.95,433.56,801.00,762.11,1386.22,1940.29,1801.58,VINATIORGA.NS
0,Close,187.92,306.10,470.16,237.62,350.42,737.80,572.70,VIPIND.NS


In [6]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis['Growth%'] = (df_gap_analysis['Mar 2023'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock,Growth%
0,Sales,2363.00,2556.00,2809.00,2765.00,2420.00,3078.00,3733.00,3MINDIA.NS,46.05
1,Profit,369.00,480.00,497.00,320.00,202.00,360.00,561.00,3MINDIA.NS,16.88
2,Sales,7.00,20.00,61.00,108.00,195.00,298.00,338.00,5PAISA.NS,1590.00
3,Profit,-16.00,-33.00,-25.00,-10.00,20.00,18.00,58.00,5PAISA.NS,-275.76
4,Sales,2903.00,3298.00,3679.00,4093.00,4310.00,4913.00,5349.00,ABBOTINDIA.NS,62.19
...,...,...,...,...,...,...,...,...,...,...
0,Close,418.32,441.52,667.82,474.70,712.50,817.36,693.90,TTKPRESTIG.NS,57.16
0,Close,NaN,NaN,NaN,26.22,29.29,14.21,24.25,UJJIVANSFB.NS,NaN
0,Close,363.95,433.56,801.00,762.11,1386.22,1940.29,1801.58,VINATIORGA.NS,315.53
0,Close,187.92,306.10,470.16,237.62,350.42,737.80,572.70,VIPIND.NS,87.10


In [11]:
stock_df_gap = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
  #print(gap_perc)
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%
0,AXISBANK.NS,3121.30
0,BAYERCROP.NS,139.98
0,CAPLIPOINT.NS,132.44
0,ICICIBANK.NS,96.80
0,LUXIND.NS,94.34
0,SIS.NS,94.31
0,MOTILALOFS.NS,85.14
0,BAJFINANCE.NS,82.77
0,SUNTV.NS,75.47
0,TEAMLEASE.NS,56.72


In [8]:
df_gap_analysis[df_gap_analysis['Stock']  == 'AXISBANK.NS']

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock,Growth%
14,Sales,26789.00,27604.00,33883.00,37996.00,34627.00,34923.00,43389.00,AXISBANK.NS,57.18
15,Profit,5954.00,566.00,7592.00,5280.00,9750.00,18929.00,18621.00,AXISBANK.NS,3189.93
0,Close,483.69,508.05,773.51,377.65,694.96,758.43,856.74,AXISBANK.NS,68.63
